In [1]:
import os
import sys
import csv
import numpy as np
from sklearn import svm
from sklearn.metrics import confusion_matrix as cmat
import collections
import itertools
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report as report  
from sklearn.model_selection import *
from sklearn.feature_selection import *
import matplotlib.pyplot as plt
import time
import random
import numpy
from datetime import datetime
import pandas as pd
import requests
import re
import xarray as xr
import pickle

In [18]:
filename = 'P26_CTDship_20170212.ctd.txt'
fieldnames = ['Pressure','Temperature','Salinity','Sigma-t','Oxygen_v','Oxygen_m','Fluorescence']
header_length = 113
ship_data = pd.read_csv(filename,names=fieldnames,header=header_length,delim_whitespace=True )

In [19]:
ship_data

,Pressure,Temperature,Salinity,Sigma-t,Oxygen_v,Oxygen_m,Fluorescence
0,23.0,5.9023,32.7213,25.7900,6.98,303.7,0.582
1,24.0,5.9024,32.7214,25.7901,6.98,304.0,0.578
2,25.1,5.9016,32.7215,25.7903,6.99,304.3,0.557
3,26.0,5.9027,32.7214,25.7901,7.01,305.0,0.560
4,26.6,5.9038,32.7212,25.7898,7.01,305.0,0.563
5,28.2,5.9031,32.7215,25.7901,7.01,305.0,0.560
6,29.0,5.9033,32.7214,25.7900,7.01,305.1,0.564
7,30.0,5.9045,32.7213,25.7898,7.00,304.7,0.554
8,30.9,5.9048,32.7212,25.7897,6.99,304.0,0.568
9,32.0,5.9043,32.7214,25.7899,6.98,304.0,0.575
